## workflow
built a cloud-based data analysis workflow with GCS + Parquet + DuckDB + Colab<BR>
pre: reason: data is too large, over 6GB, difficult to store and process, so this is the good practice that we can figure out<br>
pipeline:<br>
1. from local csv to parquet<br>
2. from local parquet to GCS parquet<br>
3. from GSC parquet to data preparation in colab, and then return back to GCS parquet<br>
E.G.Query GCS Parquet Files from Colab using DuckDB

### prepare fundamental data

In [ ]:
# STEP 1: Install and import dependencies
# --------------------------------------
!pip install duckdb gcsfs tqdm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 6.2 MB/s eta 0:00:00


In [ ]:

import duckdb
import gcsfs
import pandas as pd
from tqdm import tqdm

In [ ]:
from google.colab import auth
auth.authenticate_user()
import duckdb

query = """
SELECT *
FROM 'gs://deep_learning_final_project/model/daily/daily_model_data.parquet'
"""

df = duckdb.query(query).to_df()
df.head()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,symbol,date,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,...,SMA Value,Date_2,name,exchange,assetType,ipoDate,delistingDate,status,date_3,date_4
0,HLTH,2012-02-14,2012-02-14,0.1975,0.1975,0.1975,0.1975,0.197500,0,0.0,...,NaN,NaT,Nobilis Health Corp,NYSE,Stock,2009-02-13,NaN,Active,2012-02-14,2012-02-14
1,HLX,2012-02-14,2012-02-14,18.3400,18.5200,18.0800,18.4200,18.420000,517100,0.0,...,NaN,NaT,Helix Energy Solutions Group Inc,NYSE,Stock,1997-07-01,NaN,Active,2012-02-14,2012-02-14
2,HMC,2012-02-14,2012-02-14,35.8100,35.8900,35.6500,35.8800,25.042472,756700,0.0,...,NaN,NaT,Honda Motor,NYSE,Stock,1980-03-17,NaN,Active,2012-02-14,2012-02-14
3,HMG,2012-02-14,2012-02-14,4.0000,4.0000,3.8000,3.8000,2.047727,300,0.0,...,NaN,NaT,HMG Courtland Properties Inc,NYSE MKT,Stock,1995-08-18,NaN,Active,2012-02-14,2012-02-14
4,HMIN,2012-02-14,2012-02-14,30.7600,31.0700,29.4200,30.9900,30.990000,209926,0.0,...,NaN,NaT,Home Inns & Hotels Management Inc. ADR,NASDAQ,Stock,2006-10-26,NaN,Active,2012-02-14,2012-02-14


In [ ]:
df.columns

Index(['symbol', 'date', 'timestamp', 'open', 'high', 'low', 'close',
       'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient',
       'timestamp_1', 'Date_1', 'ATR Value', 'Upper Band', 'Middle Band',
       'Lower Band', 'BB Width', 'EMA Value', 'MACD Value', 'RSI Value',
       'SMA Value', 'Date_2', 'name', 'exchange', 'assetType', 'ipoDate',
       'delistingDate', 'status', 'date_3', 'date_4'],
      dtype='object')

In [ ]:
drop_cols = ['symbol', 'name', 'Date_1', 'Date_2', 'date_3', 'date_4', 'exchange',
             'assetType', 'ipoDate', 'delistingDate', 'status', 'dividend_amount',
             'split_coefficient', 'timestamp_1']
df.drop(columns=drop_cols, inplace=True)

# Convert timestamp to datetime (if necessary)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort by timestamp
df.sort_values(by='timestamp', inplace=True)

# Fill missing values
df.fillna(method='ffill', inplace=True)

# Define features and target
features = ['open', 'high', 'low', 'close', 'volume', 'ATR Value', 'Upper Band',
            'Middle Band', 'Lower Band', 'BB Width', 'EMA Value', 'MACD Value',
            'RSI Value', 'SMA Value']

target = 'adjusted_close'


<ipython-input-5-a5c5a8fd52c8>:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [ ]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 5.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X = df[features]
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.4f}")

RMSE: 10259611111.2881


In [ ]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Define features and target
features = ['open', 'high', 'low', 'close', 'volume', 'ATR Value', 'Upper Band',
            'Middle Band', 'Lower Band', 'BB Width', 'EMA Value', 'MACD Value',
            'RSI Value', 'SMA Value']

target = 'adjusted_close'

X = df[features].values
y = df[target].values

# Normalize data (scaling between 0 and 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

# Create sequences for LSTM (past 10 days to predict next day)
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

# Split into train/test sets
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Build LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(time_steps, X_seq.shape[2])),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=2, batch_size=32736, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions back to original scale
y_pred_rescaled = scaler.inverse_transform(y_pred)

# Print sample predictions
print("Sample Predictions:", y_pred_rescaled[:5].flatten())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
774/774 ━━━━━━━━━━━━━━━━━━━━ 305s 389ms/step - loss: 5.5992e-05 - val_loss: 1.9486e-08
Epoch 2/2
774/774 ━━━━━━━━━━━━━━━━━━━━ 290s 373ms/step - loss: 8.2191e-06 - val_loss: 3.0826e-08
197864/197864 ━━━━━━━━━━━━━━━━━━━━ 559s 3ms/step
Sample Predictions: [2.1085041e+09 2.1078551e+09 2.1076028e+09 3.9139405e+09 4.8264822e+09]
